# 02 - preprocesado

In [ ]:
%pip install seaborn
%pip install pyspellchecker
%pip install unidecode

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

#import local.lib.timeseries as ts
%matplotlib inline

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

udea-ai4eng-20242.zip: Skipping, found more recently modified local copy (use --force to force download)


## unzip and inspect data

In [ ]:
A

In [ ]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)


shape of loaded dataframe (692500, 12)


In [ ]:
z.describe(include='all')

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
count,692500.000000,692500.000000,692500,692500,686213,661643,660363,665871,669322,668836,686002,692500
unique,NaN,NaN,948,31,8,5,7,2,12,12,2,4
top,NaN,NaN,DERECHO,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Más de 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,alto
freq,NaN,NaN,53244,282159,204048,249352,232671,592514,128289,141744,382201,175619
mean,494606.130576,20198.366679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,285585.209455,10.535037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,20183.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,247324.750000,20195.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,494564.500000,20195.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,741782.500000,20203.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_EDUCACIONMADRE             668836 non-null  object
 10  ESTU_PAGOMATRICULAPROPIO        686002 non-null  object
 11  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(2), object(10)
memory

In [ ]:
# Verificar la cantidad de valores faltantes en cada columna
print("Valores nulos por columna antes de la limpieza:")
print(z.isna().sum())

Valores nulos por columna antes de la limpieza:
ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_EDUCACIONMADRE               23664
ESTU_PAGOMATRICULAPROPIO           6498
RENDIMIENTO_GLOBAL                    0
dtype: int64


In [ ]:


# Para las columnas de texto:
z['FAMI_EDUCACIONMADRE'].values[z['FAMI_EDUCACIONMADRE'].isna()] = 'no info'
z['ESTU_VALORMATRICULAUNIVERSIDAD'].values[z['ESTU_VALORMATRICULAUNIVERSIDAD'].isna()] = 'no info'
z['FAMI_ESTRATOVIVIENDA'].values[z['FAMI_ESTRATOVIVIENDA'].isna()] = 'no info'
z['FAMI_TIENEINTERNET'].values[z['FAMI_TIENEINTERNET'].isna()] = 'no info'
z['FAMI_EDUCACIONPADRE'].values[z['FAMI_EDUCACIONPADRE'].isna()] = 'no info'


z['ESTU_HORASSEMANATRABAJA'].values[z['ESTU_HORASSEMANATRABAJA'].isna()] = '0'


z['ESTU_PAGOMATRICULAPROPIO'].values[z['ESTU_PAGOMATRICULAPROPIO'].isna()] = 'No'


print("Valores nulos por columna después de la limpieza:")
print(z.isna().sum())

Valores nulos por columna después de la limpieza:
ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


In [ ]:
agrupado_periodo = z.groupby('PERIODO', dropna=False).size()
print("Agrupación por PERIODO:")
print(agrupado_periodo)

Agrupación por PERIODO:
PERIODO
20183    164818
20184       254
20194      1472
20195    180873
20196       165
20202       490
20203    171838
20212    171412
20213      1178
dtype: int64


In [ ]:
agrupado_programa_academico = z.groupby('ESTU_PRGM_ACADEMICO', dropna=False).size()
print("\nAgrupación por ESTU_PRGM_ACADEMICO:")
print(agrupado_programa_academico)


Agrupación por ESTU_PRGM_ACADEMICO:
ESTU_PRGM_ACADEMICO
3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES       69
ACTIVIDAD FISICA Y DEPORTE                          162
ACUICULTURA                                          79
ADMINISTRACION                                      117
ADMINISTRACION  FINANCIERA                           74
                                                  ...  
TRABAJO SOCIAL                                    12686
TRADUCCION INGLES-FRANCES-ESPAÑOL                    68
TURISMO                                             222
URBANISMO                                             7
ZOOTECNIA                                          1669
Length: 948, dtype: int64


In [ ]:
agrupado_departamento = z.groupby('ESTU_PRGM_DEPARTAMENTO', dropna=False).size()
print("\nAgrupación por ESTU_PRGM_DEPARTAMENTO:")
print(agrupado_departamento)


Agrupación por ESTU_PRGM_DEPARTAMENTO:
ESTU_PRGM_DEPARTAMENTO
AMAZONAS               40
ANTIOQUIA           83607
ARAUCA                589
ATLANTICO           41020
BOGOTÁ             282159
BOLIVAR             20629
BOYACA              14048
CALDAS              11640
CAQUETA              2659
CASANARE             1852
CAUCA               11471
CESAR                8279
CHOCO                4289
CORDOBA             12188
CUNDINAMARCA        14018
GUAVIARE               37
HUILA                9995
LA GUAJIRA           4778
MAGDALENA            9512
META                 7910
NARIÑO              13454
NORTE SANTANDER     22588
PUTUMAYO              795
QUINDIO              8229
RISARALDA           12679
SAN ANDRES             10
SANTANDER           28828
SUCRE                8674
TOLIMA              11921
VALLE               44588
VAUPES                 14
dtype: int64


In [ ]:
agrupado_valor_matricula = z.groupby('ESTU_VALORMATRICULAUNIVERSIDAD', dropna=False).size()
print("\nAgrupación por ESTU_VALORMATRICULAUNIVERSIDAD:")
print(agrupado_valor_matricula)


Agrupación por ESTU_VALORMATRICULAUNIVERSIDAD:
ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      204048
Entre 2.5 millones y menos de 4 millones    127430
Entre 4 millones y menos de 5.5 millones     69736
Entre 5.5 millones y menos de 7 millones     38490
Entre 500 mil y menos de 1 millón            78704
Menos de 500 mil                             80263
Más de 7 millones                            68014
No pagó matrícula                            19528
no info                                       6287
dtype: int64


In [ ]:
# Aplicar el mapeo utilizando map
z["ESTU_VALORMATRICULAUNIVERSIDAD"] = z["ESTU_VALORMATRICULAUNIVERSIDAD"].map({'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Menos de 500 mil': .250,
 'Entre 500 mil y menos de 1 millón': .75,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Más de 7 millones': 7.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'No pagó matrícula': 0,
 'no info': -1})

# Verificar los resultados
z["ESTU_VALORMATRICULAUNIVERSIDAD"]

,ESTU_VALORMATRICULAUNIVERSIDAD
0,6.25
1,3.25
2,3.25
3,4.75
4,3.25
...,...
692495,0.75
692496,3.25
692497,1.75
692498,3.25


In [ ]:
agrupado_horas_trabajo = z.groupby('ESTU_HORASSEMANATRABAJA', dropna=False).size()
print("\nAgrupación por ESTU_HORASSEMANATRABAJA:")
print(agrupado_horas_trabajo)


Agrupación por ESTU_HORASSEMANATRABAJA:
ESTU_HORASSEMANATRABAJA
0                      147407
Entre 11 y 20 horas    115857
Entre 21 y 30 horas     92693
Menos de 10 horas       87191
Más de 30 horas        249352
dtype: int64


In [ ]:
# Aplicar el mapeo utilizando map
z["ESTU_HORASSEMANATRABAJA"] = z["ESTU_HORASSEMANATRABAJA"].map({'0': 0,
    'Menos de 10 horas': 5,
    'Entre 11 y 20 horas': 15,
    'Entre 21 y 30 horas': 25,
    'Más de 30 horas': 35,
    'No aplica': -1,
    'no info': -1,})

# Verificar los resultados
z["ESTU_HORASSEMANATRABAJA"]

,ESTU_HORASSEMANATRABAJA
0,5
1,0
2,35
3,0
4,25
...,...
692495,15
692496,35
692497,5
692498,5


In [ ]:
agrupado_estrato_vivienda = z.groupby('FAMI_ESTRATOVIVIENDA', dropna=False).size()
print("\nAgrupación por FAMI_ESTRATOVIVIENDA:")
print(agrupado_estrato_vivienda)


Agrupación por FAMI_ESTRATOVIVIENDA:
FAMI_ESTRATOVIVIENDA
Estrato 1      111991
Estrato 2      232671
Estrato 3      210685
Estrato 4       65514
Estrato 5       23608
Estrato 6       12605
Sin Estrato      3289
no info         32137
dtype: int64


In [ ]:


# Aplicar el mapeo utilizando map
z["FAMI_ESTRATOVIVIENDA"] = z["FAMI_ESTRATOVIVIENDA"].map({'Estrato 1': 1,'Estrato 2': 2,'Estrato 3': 3,'Estrato 4': 4,'Estrato 5': 5,
'Estrato 6': 6,"Sin Estrato":0})

# Verificar los resultados
z["FAMI_ESTRATOVIVIENDA"]




,FAMI_ESTRATOVIVIENDA
0,3.0
1,3.0
2,3.0
3,4.0
4,3.0
...,...
692495,2.0
692496,3.0
692497,3.0
692498,1.0


In [ ]:
agrupado_internet = z.groupby('FAMI_TIENEINTERNET', dropna=False).size()
print("\nAgrupación por FAMI_TIENEINTERNET:")
print(agrupado_internet)


Agrupación por FAMI_TIENEINTERNET:
FAMI_TIENEINTERNET
No          73357
Si         592514
no info     26629
dtype: int64


In [ ]:

# Aplicar el mapeo utilizando map
z["FAMI_TIENEINTERNET"] = z["FAMI_TIENEINTERNET"].map({'Si': 1,
    'No': 0, })

# Verificar los resultados
z["FAMI_TIENEINTERNET"]


,FAMI_TIENEINTERNET
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
692495,1.0
692496,1.0
692497,1.0
692498,0.0


In [ ]:
agrupado_educacion_padre = z.groupby('FAMI_EDUCACIONPADRE', dropna=False).size()
print("\nAgrupación por FAMI_EDUCACIONPADRE:")
print(agrupado_educacion_padre)


Agrupación por FAMI_EDUCACIONPADRE:
FAMI_EDUCACIONPADRE
Educación profesional completa           83117
Educación profesional incompleta         27084
Ninguno                                  22008
No Aplica                                 9229
No sabe                                  16592
Postgrado                                44169
Primaria completa                        55958
Primaria incompleta                     125675
Secundaria (Bachillerato) completa      128289
Secundaria (Bachillerato) incompleta     71654
Técnica o tecnológica completa           62995
Técnica o tecnológica incompleta         22552
no info                                  23178
dtype: int64


In [ ]:
z = z.copy()
z.FAMI_EDUCACIONMADRE = ['no info' if i in ['No sabe', 'No Aplica'] else i for i in z.FAMI_EDUCACIONMADRE.values]

In [ ]:
# Aplicar el mapeo utilizando map
z["FAMI_EDUCACIONPADRE"] = z["FAMI_EDUCACIONPADRE"].map({'Ninguno': 0,
    'Primaria incompleta': 1,
    'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3,
    'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 5,
    'Técnica o tecnológica completa': 6,
    'Educación profesional incompleta': 7,
    'Educación profesional completa': 8,
    'Postgrado': 9,
    'No sabe': -1,
    'No Aplica': -1,
    'no info': -1,})

# Verificar los resultados
z["FAMI_EDUCACIONPADRE"]

,FAMI_EDUCACIONPADRE
0,5
1,6
2,4
3,-1
4,2
...,...
692495,4
692496,1
692497,4
692498,2


In [ ]:
agrupado_educacion_madre = z.groupby('FAMI_EDUCACIONMADRE', dropna=False).size()
print("\nAgrupación por FAMI_EDUCACIONMADRE:")
print(agrupado_educacion_madre)


Agrupación por FAMI_EDUCACIONMADRE:
FAMI_EDUCACIONMADRE
Educación profesional completa           85326
Educación profesional incompleta         22470
Ninguno                                  14483
Postgrado                                46246
Primaria completa                        56125
Primaria incompleta                      99420
Secundaria (Bachillerato) completa      141744
Secundaria (Bachillerato) incompleta     81012
Técnica o tecnológica completa           89542
Técnica o tecnológica incompleta         27533
no info                                  28599
dtype: int64


In [ ]:
# Aplicar el mapeo utilizando map
z["FAMI_EDUCACIONMADRE"] = z["FAMI_EDUCACIONMADRE"].map({'Ninguno': 0,
    'Primaria incompleta': 1,
    'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3,
    'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 5,
    'Técnica o tecnológica completa': 6,
    'Educación profesional incompleta': 7,
    'Educación profesional completa': 8,
    'Postgrado': 9,
    'No sabe': -1,
    'No Aplica': -1,
    'no info': -1,})

# Verificar los resultados
z["FAMI_EDUCACIONMADRE"]

,FAMI_EDUCACIONMADRE
0,9
1,5
2,4
3,4
4,2
...,...
692495,3
692496,3
692497,3
692498,2


In [ ]:
agrupado_pago_propio = z.groupby('ESTU_PAGOMATRICULAPROPIO', dropna=False).size()
print("\nAgrupación por ESTU_PAGOMATRICULAPROPIO:")
print(agrupado_pago_propio)


Agrupación por ESTU_PAGOMATRICULAPROPIO:
ESTU_PAGOMATRICULAPROPIO
No    388699
Si    303801
dtype: int64


In [ ]:
# Aplicar el mapeo utilizando map
z["ESTU_PAGOMATRICULAPROPIO"] = z["ESTU_PAGOMATRICULAPROPIO"].map({'No': 0,'Si': 1,})

# Verificar los resultados
z["ESTU_PAGOMATRICULAPROPIO"]

,ESTU_PAGOMATRICULAPROPIO
0,0
1,0
2,0
3,0
4,0
...,...
692495,1
692496,0
692497,1
692498,1


In [ ]:
agrupado_rendimiento_global = z.groupby('RENDIMIENTO_GLOBAL', dropna=False).size()
print("\nAgrupación por RENDIMIENTO_GLOBAL:")
print(agrupado_rendimiento_global)


Agrupación por RENDIMIENTO_GLOBAL:
RENDIMIENTO_GLOBAL
alto          175619
bajo          172987
medio-alto    171619
medio-bajo    172275
dtype: int64


In [ ]:
# Aplicar el mapeo utilizando map
z["RENDIMIENTO_GLOBAL"] = z["RENDIMIENTO_GLOBAL"].map({'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2,})

# Verificar los resultados
z["RENDIMIENTO_GLOBAL"]

,RENDIMIENTO_GLOBAL
0,2
1,0
2,0
3,3
4,1
...,...
692495,2
692496,0
692497,1
692498,0


In [ ]:
z

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,6.25,5,3.0,1.0,5,9,0,2
1,645256,20212,DERECHO,ATLANTICO,3.25,0,3.0,0.0,6,5,0,0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3.25,35,3.0,1.0,4,4,0,0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4.75,0,4.0,1.0,-1,4,0,3
4,989032,20212,PSICOLOGIA,ANTIOQUIA,3.25,25,3.0,1.0,2,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,0.75,15,2.0,1.0,4,3,1,2
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,3.25,35,3.0,1.0,1,3,0,0
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,1.75,5,3.0,1.0,4,3,1,1
692498,986620,20195,PSICOLOGIA,TOLIMA,3.25,5,1.0,0.0,2,2,1,0


In [ ]:
# Valores únicos en columnas categóricas
for col in z:
    unique_values = z[col].unique()
    print(f'{col} has {len(unique_values)} unique values: {unique_values}')

ID has 692500 unique values: [904256 645256 308367 ... 504185 986620 933374]
PERIODO has 9 unique values: [20212 20203 20195 20183 20194 20213 20184 20202 20196]
ESTU_PRGM_ACADEMICO has 948 unique values: ['ENFERMERIA' 'DERECHO' 'MERCADEO Y PUBLICIDAD'
 'ADMINISTRACION DE EMPRESAS' 'PSICOLOGIA' 'MEDICINA VETERINARIA'
 'INGENIERIA MECANICA' 'ADMINISTRACIÓN EN SALUD OCUPACIONAL'
 'INGENIERIA INDUSTRIAL' 'ADMINISTRACIÓN FINANCIERA' 'HOTELERIA Y TURISMO'
 'LICENCIATURA EN CIENCIAS SOCIALES' 'LICENCIATURA EN PEDAGOGIA INFANTIL'
 'COMUNICACION SOCIAL' 'CIENCIA POLITICA'
 'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL'
 'MAESTRO EN MÚSICA' 'INGENIERIA MECATRONICA' 'TRABAJO SOCIAL'
 'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL' 'INGENIERIA CIVIL'
 'CONTADURIA PÚBLICA' 'ADMINISTRACION EN SALUD'
 'ADMINISTRACIÓN DE EMPRESAS' 'ESTADISTICA' 'LICENCIATURA EN BIOLOGIA'
 'INGENIERIA AGROINDUSTRIAL' 'ZOOTECNIA' 'COMUNICACION AUDIOVISUAL'
 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN

In [ ]:
# Limpiar la columna 'ESTU_PRGM_ACADEMICO'
from spellchecker import SpellChecker
from unidecode import unidecode

# Limpiar la columna 'ESTU_PRGM_ACADEMICO'
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].str.strip()
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].str.replace(',', '')
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].str.replace('-', '')
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].str.replace('.', '')
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].str.lower()
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].str.normalize('NFKD')
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].str.replace('[áéíóúÁÉÍÓÚ]', 'aeiouAEIOU')
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].apply(unidecode)

def column_cleaning(df, column):
    df[column] = df[column].str.strip()
    df[column] = df[column].str.replace(',', '')
    df[column] = df[column].str.replace('-', '')
    df[column] = df[column].str.replace('.', '')
    df[column] = df[column].str.lower()
    df[column] = df[column].str.normalize('NFKD')
    df[column] = df[column].str.replace('[áéíóúÁÉÍÓÚ]', 'aeiouAEIOU')
    df[column] = df[column].apply(unidecode)
    return df

spell = SpellChecker(language=None)  # Establecer el idioma en español

def correct_spelling(text):
    # Separar el texto en palabras
    words = text.split()
    # Corregir ortografía para cada palabra
    corrected_words = []
    for word in words:
        corrected_word = spell.correction(word)
        # Verificar si la palabra corregida es no nula y no contiene caracteres especiales
        if corrected_word is not None and not any(char in corrected_word for char in ['?', '¿', '¡', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '-', '+', '=', '{', '}', '[', ']', '|', '\\', '/', ':', ';', '"', "'", '<', '>', ',', '.']):
            corrected_words.append(corrected_word)
    # Unir las palabras corregidas nuevamente en un solo texto
    corrected_text = ' '.join(corrected_words)
    return corrected_text


# Aplicar la corrección ortográfica a toda la columna
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].apply(correct_spelling)

# Eliminar valores duplicados
unique_values = z['ESTU_PRGM_ACADEMICO'].unique()
print(len(unique_values))
#print(unique_values)

714


In [ ]:
missing_values = z.isnull().sum()
print(missing_values)

ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD        0
ESTU_HORASSEMANATRABAJA               0
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE                   0
FAMI_EDUCACIONMADRE                   0
ESTU_PAGOMATRICULAPROPIO              0
RENDIMIENTO_GLOBAL                    0
dtype: int64


In [ ]:
# Rellenar valores faltantes con un valor fijo
z['FAMI_ESTRATOVIVIENDA'].fillna('Desconocido', inplace=True)
z['FAMI_TIENEINTERNET'].fillna('No especificado', inplace=True)


<ipython-input-149-8d55145171ec>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  z['FAMI_ESTRATOVIVIENDA'].fillna('Desconocido', inplace=True)
<ipython-input-149-8d55145171ec>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Desconocido' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  z['FAMI_ESTRATOVIVIENDA'].fillna('Desconocido', inplace=True)
<ipython-input-149-8d55145171ec>:3: FutureWarning: A valu

In [ ]:
missing_values = z.isnull().sum()
print(missing_values)

ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


In [ ]:
import re

# Definir la función para categorizar las carreras
def categorize_career(career_name):
    career_lower = career_name.lower()
    if re.search(r'ingenieria', career_lower):
        return 'Ingeniería'
    elif re.search(r'administracion|mercadeo|negocios|banca|finanzas', career_lower):
        return 'Administración y Negocios'
    elif re.search(r'ciencias exactas|matematicas|estadistica|fisica|quimica|geologia', career_lower):
        return 'Ciencias Exactas'
    elif re.search(r'medicina|enfermeria|veterinaria|odontologia|terapia ocupacional|bacteriologia|fonoaudiologia|terapia respiratoria|microbiologia', career_lower):
        return 'Ciencias de la Salud'
    elif re.search(r'comunicacion social|periodismo|comunicacion grafica|comunicacion digital|arqueologia|comunicacion entretenimiento digital|comunicacion periodismo', career_lower):
        return 'Comunicación y Periodismo'
    elif re.search(r'derecho|criminalistica|arquitectura', career_lower):
        return 'Derecho y Arquitectura'
    elif re.search(r'licenciatura|educacion|pedagogia|lenguas|artes|musica|filosofia|danza|teologia|letras|bibliotecologia|antropologia|geografia|diseno', career_lower):
        return 'Licenciaturas y Educación'
    elif re.search(r'software|informatica|telematica', career_lower):
        return 'Tecnología de la Información'
    elif re.search(r'diseno|artes visuales|diseno digital', career_lower):
        return 'Diseño y Artes Visuales'
    elif re.search(r'deporte|actividad fisica|cultura fisica', career_lower):
        return 'Deporte y Actividad Física'
    elif re.search(r'gastro|gastronomia|cocina', career_lower):
        return 'Gastronomía'
    else:
        return 'Otro'

# Aplicar la función categorize_career a la columna ESTU_PRGM_ACADEMICO y crear la columna Sector_Estudio
z['ESTU_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO'].apply(categorize_career)

unique_values = z['ESTU_PRGM_ACADEMICO'].unique()
print(unique_values)
#print(unique_values)
z['ESTU_PRGM_ACADEMICO']

['Ciencias de la Salud' 'Derecho y Arquitectura'
 'Administración y Negocios' 'Otro' 'Ingeniería'
 'Licenciaturas y Educación' 'Comunicación y Periodismo'
 'Ciencias Exactas' 'Deporte y Actividad Física'
 'Tecnología de la Información' 'Gastronomía']


,ESTU_PRGM_ACADEMICO
0,Ciencias de la Salud
1,Derecho y Arquitectura
2,Administración y Negocios
3,Administración y Negocios
4,Otro
...,...
692495,Otro
692496,Otro
692497,Administración y Negocios
692498,Otro


In [ ]:
z.describe(include='all')

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
count,692500.000000,692500.000000,692500,692500,692500.000000,692500.000000,692500.0,692500.0,692500.000000,692500.000000,692500.000000,692500.000000
unique,NaN,NaN,11,31,NaN,NaN,8.0,3.0,NaN,NaN,NaN,NaN
top,NaN,NaN,Otro,BOGOTÁ,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN
freq,NaN,NaN,172537,282159,NaN,NaN,232671.0,592514.0,NaN,NaN,NaN,NaN
mean,494606.130576,20198.366679,NaN,NaN,2.805712,19.088022,NaN,NaN,3.840412,4.222544,0.438702,1.505227
std,285585.209455,10.535037,NaN,NaN,2.323047,14.184090,NaN,NaN,2.928368,2.746051,0.496229,1.121061
min,1.000000,20183.000000,NaN,NaN,-1.000000,0.000000,NaN,NaN,-1.000000,-1.000000,0.000000,0.000000
25%,247324.750000,20195.000000,NaN,NaN,0.750000,5.000000,NaN,NaN,1.000000,2.000000,0.000000,1.000000
50%,494564.500000,20195.000000,NaN,NaN,1.750000,15.000000,NaN,NaN,4.000000,4.000000,0.000000,2.000000
75%,741782.500000,20203.000000,NaN,NaN,4.750000,35.000000,NaN,NaN,6.000000,6.000000,1.000000,3.000000


In [ ]:
drop_cols = ['PERIODO','ID']
for i in drop_cols:
  z.drop(i,
  axis='columns', inplace=True)

In [ ]:
z.head()

,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,Ciencias de la Salud,BOGOTÁ,6.25,5,3.0,1.0,5,9,0,2
1,Derecho y Arquitectura,ATLANTICO,3.25,0,3.0,0.0,6,5,0,0
2,Administración y Negocios,BOGOTÁ,3.25,35,3.0,1.0,4,4,0,0
3,Administración y Negocios,SANTANDER,4.75,0,4.0,1.0,-1,4,0,3
4,Otro,ANTIOQUIA,3.25,25,3.0,1.0,2,2,0,1


In [ ]:
df_encoded = pd.get_dummies(z, columns = [
    'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'], dtype=int)

df_encoded.head()


,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL,ESTU_PRGM_ACADEMICO_Administración y Negocios,ESTU_PRGM_ACADEMICO_Ciencias Exactas,ESTU_PRGM_ACADEMICO_Ciencias de la Salud,ESTU_PRGM_ACADEMICO_Comunicación y Periodismo,ESTU_PRGM_ACADEMICO_Deporte y Actividad Física,ESTU_PRGM_ACADEMICO_Derecho y Arquitectura,ESTU_PRGM_ACADEMICO_Gastronomía,ESTU_PRGM_ACADEMICO_Ingeniería,ESTU_PRGM_ACADEMICO_Licenciaturas y Educación,ESTU_PRGM_ACADEMICO_Otro,ESTU_PRGM_ACADEMICO_Tecnología de la Información,ESTU_PRGM_DEPARTAMENTO_AMAZONAS,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES
0,6.25,5,3.0,1.0,5,9,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3.25,0,3.0,0.0,6,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3.25,35,3.0,1.0,4,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4.75,0,4.0,1.0,-1,4,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,3.25,25,3.0,1.0,2,2,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_encoded.columns

Index(['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO', 'RENDIMIENTO_GLOBAL',
       'ESTU_PRGM_ACADEMICO_Administración y Negocios',
       'ESTU_PRGM_ACADEMICO_Ciencias Exactas',
       'ESTU_PRGM_ACADEMICO_Ciencias de la Salud',
       'ESTU_PRGM_ACADEMICO_Comunicación y Periodismo',
       'ESTU_PRGM_ACADEMICO_Deporte y Actividad Física',
       'ESTU_PRGM_ACADEMICO_Derecho y Arquitectura',
       'ESTU_PRGM_ACADEMICO_Gastronomía', 'ESTU_PRGM_ACADEMICO_Ingeniería',
       'ESTU_PRGM_ACADEMICO_Licenciaturas y Educación',
       'ESTU_PRGM_ACADEMICO_Otro',
       'ESTU_PRGM_ACADEMICO_Tecnología de la Información',
       'ESTU_PRGM_DEPARTAMENTO_AMAZONAS', 'ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA',
       'ESTU_PRGM_DEPARTAMENTO_ARAUCA', 'ESTU_PRGM_DEPARTAMENTO_ATLANTICO',
       'ESTU_PRGM_DEPARTAMENTO_BOGOTÁ', 'ESTU_PRGM_DEPARTAMENTO_BOLIVAR',
  

In [ ]:
df_encoded.to_csv('df_filtrado.csv')

In [ ]:
df_rendimiento = df_encoded['RENDIMIENTO_GLOBAL'].to_list()
df_encoded.drop('RENDIMIENTO_GLOBAL',
  axis='columns', inplace=True)
df_rendimiento

[2,
 0,
 0,
 3,
 1,
 2,
 3,
 1,
 1,
 3,
 0,
 1,
 1,
 1,
 1,
 3,
 2,
 1,
 3,
 0,
 3,
 1,
 0,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 3,
 3,
 0,
 2,
 3,
 1,
 0,
 2,
 2,
 1,
 2,
 2,
 3,
 0,
 0,
 0,
 0,
 0,
 3,
 2,
 2,
 1,
 1,
 1,
 0,
 3,
 3,
 2,
 1,
 2,
 2,
 3,
 2,
 1,
 1,
 0,
 2,
 2,
 2,
 1,
 2,
 3,
 2,
 0,
 3,
 0,
 1,
 3,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 3,
 1,
 3,
 0,
 3,
 1,
 3,
 2,
 2,
 3,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 3,
 1,
 1,
 1,
 2,
 3,
 2,
 0,
 1,
 3,
 2,
 0,
 2,
 0,
 3,
 3,
 2,
 1,
 0,
 3,
 1,
 2,
 0,
 3,
 3,
 3,
 0,
 1,
 0,
 0,
 1,
 3,
 0,
 3,
 1,
 3,
 3,
 3,
 1,
 3,
 2,
 1,
 2,
 2,
 3,
 1,
 3,
 1,
 1,
 3,
 1,
 3,
 1,
 2,
 3,
 3,
 2,
 0,
 3,
 0,
 2,
 2,
 0,
 0,
 2,
 3,
 1,
 1,
 2,
 3,
 3,
 2,
 3,
 1,
 3,
 1,
 3,
 1,
 2,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 3,
 3,
 0,
 0,
 3,
 1,
 2,
 2,
 1,
 0,
 2,
 3,
 1,
 1,
 2,
 3,
 0,
 3,
 2,
 1,
 0,
 3,
 2,
 0,
 1,
 3,
 0,
 3,
 3,
 1,
 3,
 1,
 0,
 3,
 0,
 2,
 3,
 3,
 0,
 0,
 3,
 1,
 3,
 1,
 1,
 2,


In [168]:
# Dividir datos en entrenamiento y prueba
X = df_encoded
y = df_rendimiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [170]:
non_numeric_columns = X.select_dtypes(include=['object']).columns
print(f"Columnas no numéricas: {list(non_numeric_columns)}")


Columnas no numéricas: ['FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET']


In [173]:
# Convertir columnas categóricas a numéricas usando get_dummies
X = pd.get_dummies(X, columns=['FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET'], drop_first=True)
# Confirmar que todas las columnas son numéricas
print("Tipos de datos después de la codificación:")
print(X_train.dtypes)

KeyError: "None of [Index(['FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET'], dtype='object')] are in the [columns]"

In [172]:
# Entrenar modelo de Regresión Logística
model = LogisticRegression(max_iter=500, random_state=42)
model.fit(X_train, y_train)



ValueError: could not convert string to float: 'Desconocido'

In [ ]:
# Predicciones
y_pred = model.predict(X_test)

# Evaluación del modelo
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusión - Logistic Regression')
plt.show()